<a target="_blank" href="https://colab.research.google.com/github/sonder-art/fdd_o23/blob/main/codigo/polars/04_ejercicio_polars.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

In [2]:
!pip install polars

Defaulting to user installation because normal site-packages is not writeable
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.1/30.1 MB 4.5 MB/s eta 0:00:0000:0100:01

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: /Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip


In [3]:
import polars as pl
import random
import string


# Generacion de Datos

In [6]:
# Funciones auxiliares para generar datos
def random_string(length=5):
    letters = string.ascii_lowercase
    return ''.join(random.choice(letters) for i in range(length))

def generate_data(n=1000):
    data = {
        "id_venta": list(range(1, n + 1)),
        "producto": [random_string() for _ in range(n)],
        "cantidad": [random.randint(1, 20) for _ in range(n)],
        "precio_unitario": [random.uniform(10, 100) for _ in range(n)],
        "fecha_venta": [pl.date(random.randint(2010, 2022), random.randint(1, 12), random.randint(1, 28)) for _ in range(n)],
        "cliente_id": [random.randint(1000, 5000) for _ in range(n)]
    }
    return pl.DataFrame(data)

# Generar el DataFrame
df = generate_data()
df.head()

id_venta,producto,cantidad,precio_unitario,fecha_venta,cliente_id
i64,str,i64,f64,object,i64
1,"""dlzom""",18,54.630622,"2015-10-14 00:00:00.alias(""datetime"").strict_cast(Date).alias(""date"")",1914
2,"""gdomh""",2,87.946336,"2011-04-10 00:00:00.alias(""datetime"").strict_cast(Date).alias(""date"")",4134
3,"""ljnkm""",3,66.622685,"2020-03-25 00:00:00.alias(""datetime"").strict_cast(Date).alias(""date"")",4943
4,"""bxvel""",16,32.152501,"2015-01-28 00:00:00.alias(""datetime"").strict_cast(Date).alias(""date"")",1599
5,"""wfhos""",10,50.654158,"2010-05-03 00:00:00.alias(""datetime"").strict_cast(Date).alias(""date"")",1176


In [7]:
df = df.lazy()
df

# Limpieza de Datos

Utilizando Polars en modo Lazy, realice las siguientes operaciones de limpieza y manipulación de datos:

    1. `Filtrar` las ventas con cantidades menores a 5.
    2. `Agregar` una nueva columna llamada total_venta, que sea el resultado de multiplicar cantidad por precio_unitario.
    3. `Eliminar` la columna precio_unitario después de crear total_venta.
    4. `Agrupar` los datos por producto y calcular el total vendido y la cantidad total vendida por producto.
    5. `Ordenar` los resultados por el total vendido en orden descendente.

Todo esto lo debes de realizar en una sola linea de codigo

In [24]:

lazy_df = df.lazy()

# Realizar las operaciones de limpieza y manipulación
resultado = (
    lazy_df
    .filter(pl.col("cantidad") < 5) # 1
    .with_columns([
        (pl.col("cantidad") * pl.col("precio_unitario")).alias("total_venta") # 2
    ])
    .drop("precio_unitario") # 3
    .group_by("producto")
    .agg([
        pl.sum("total_venta").alias("total_vendido"), # 4
        pl.sum("cantidad").alias("cantidad_total_vendida")
    ])
    .sort("total_vendido", descending=True) # 5
)

# Ejecutar y mostrar los resultados
resultado_df = resultado.collect()
resultado_df

producto,total_vendido,cantidad_total_vendida
str,f64,i64
"""ijenf""",391.488223,4
"""oagch""",391.097361,4
"""rbqdw""",387.015869,4
"""jopgf""",358.665751,4
"""zaglg""",347.515665,4
…,…,…
"""fwusk""",22.968338,1
"""bgwqw""",20.910128,1
"""gsguf""",20.712569,2


# Guarda el resultado en un archivo `parquet`

Guarda el resultado en un archivo parquet que se llame `resultados.parquet`

In [25]:
# Tu codigo aqui
resultado_df.write_parquet("resultado.parquet")

# Lee Parquet

Crea codigo para leer en polars el archivo parquet

In [26]:
# Tu codigo aqui
pl.read_parquet("resultado.parquet")


producto,total_vendido,cantidad_total_vendida
str,f64,i64
"""ijenf""",391.488223,4
"""oagch""",391.097361,4
"""rbqdw""",387.015869,4
"""jopgf""",358.665751,4
"""zaglg""",347.515665,4
…,…,…
"""fwusk""",22.968338,1
"""bgwqw""",20.910128,1
"""gsguf""",20.712569,2
